[View in Colaboratory](https://colab.research.google.com/github/InonS/Open-GPGPU-ANN/blob/master/Google_Colaboratory_TensorFlow_with_GPU.ipynb)

# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [2]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
9.763288062000356
GPU (s):
0.6810243039999477
GPU speedup over CPU: 14x


# Benchmark TensorFlow GPU on real ANN

## Download models

In [0]:
models = {"mobilenet_v2_1.0_96": "https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_96.tgz",
          "resnet_v2_101_2017_04_14": "http://download.tensorflow.org/models/resnet_v2_101_2017_04_14.tar.gz"}

In [4]:
!wget https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_96.tgz && \
    wget http://download.tensorflow.org/models/resnet_v2_101_2017_04_14.tar.gz

!ls -sh1


Redirecting output to ‘wget-log.6’.

Redirecting output to ‘wget-log.7’.
total 1.1G
4.0K mobilenet_v2_1.0_96
 75M mobilenet_v2_1.0_96.tgz
 75M mobilenet_v2_1.0_96.tgz.1
4.0K resnet_v2_101_2017_04_14
477M resnet_v2_101_2017_04_14.tar.gz
477M resnet_v2_101_2017_04_14.tar.gz.1
196K resnet_v2.py
216K resnet_v2.py.1
4.0K sample_data
4.0K wget-log
4.0K wget-log.1
4.0K wget-log.2
4.0K wget-log.3
4.0K wget-log.4
4.0K wget-log.5
4.0K wget-log.6
4.0K wget-log.7


In [5]:
!mkdir mobilenet_v2_1.0_96 && cd mobilenet_v2_1.0_96 && tar fvxz ../mobilenet_v2_1.0_96.tgz && cd .. && rm mobilenet_v2_1.0_96.tgz && \
mkdir resnet_v2_101_2017_04_14 && cd resnet_v2_101_2017_04_14 && tar fvxz ../resnet_v2_101_2017_04_14.tar.gz && cd .. && rm resnet_v2_101_2017_04_14.tar.gz

mkdir: cannot create directory ‘mobilenet_v2_1.0_96’: File exists


In [6]:
!ls -R

.:
mobilenet_v2_1.0_96		   resnet_v2.py    wget-log.3
mobilenet_v2_1.0_96.tgz		   resnet_v2.py.1  wget-log.4
mobilenet_v2_1.0_96.tgz.1	   sample_data	   wget-log.5
resnet_v2_101_2017_04_14	   wget-log	   wget-log.6
resnet_v2_101_2017_04_14.tar.gz    wget-log.1	   wget-log.7
resnet_v2_101_2017_04_14.tar.gz.1  wget-log.2

./mobilenet_v2_1.0_96:
mobilenet_v2_1.0_96.ckpt.data-00000-of-00001  mobilenet_v2_1.0_96_frozen.pb
mobilenet_v2_1.0_96.ckpt.index		      mobilenet_v2_1.0_96_info.txt
mobilenet_v2_1.0_96.ckpt.meta		      mobilenet_v2_1.0_96.tflite
mobilenet_v2_1.0_96_eval.pbtxt

./resnet_v2_101_2017_04_14:
eval.graph  resnet_v2_101.ckpt	train.graph

./sample_data:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


## Sample data

In [7]:
!ls sample_data

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


### MNIST

In [8]:
from pandas import read_csv

mnist_train_small = read_csv("sample_data/mnist_train_small.csv", index_col=False)
mnist_train_small.shape

(19999, 785)

In [9]:
mnist_train_small.info()
mnist_train_small.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Columns: 785 entries, 6 to 0.590
dtypes: int64(785)
memory usage: 119.8 MB


,6,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.581,0.582,0.583,0.584,0.585,0.586,0.587,0.588,0.589,0.590
count,19999.000000,19999.0,19999.0,19999.0,19999.0,19999.0,19999.0,19999.0,19999.0,19999.0,...,19999.000000,19999.000000,19999.000000,19999.000000,19999.000000,19999.000000,19999.0,19999.0,19999.0,19999.0
mean,4.470124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.215011,0.087704,0.036502,0.013651,0.032602,0.006000,0.0,0.0,0.0,0.0
std,2.892807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.314821,3.921664,2.712527,0.950818,2.718102,0.600333,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,253.000000,254.000000,253.000000,79.000000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [10]:
from pandas import Series

y_mnist_train_small = Series([sample[0] for sample in mnist_train_small.values])
y_mnist_train_small.describe()

count    19999.000000
mean         4.470124
std          2.892807
min          0.000000
25%          2.000000
50%          4.000000
75%          7.000000
max          9.000000
dtype: float64

In [11]:
n_classes = len(y_mnist_train_small.unique())
n_classes

10

In [12]:
from pandas import DataFrame

x_mnist_train_small = DataFrame([sample for sample in mnist_train_small.values[:,1:]])
x_mnist_train_small.info()
x_mnist_train_small.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Columns: 784 entries, 0 to 783
dtypes: int64(784)
memory usage: 119.6 MB


,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
count,19999.0,19999.0,19999.0,19999.0,19999.0,19999.0,19999.0,19999.0,19999.0,19999.0,...,19999.000000,19999.000000,19999.000000,19999.000000,19999.000000,19999.000000,19999.0,19999.0,19999.0,19999.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.215011,0.087704,0.036502,0.013651,0.032602,0.006000,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.314821,3.921664,2.712527,0.950818,2.718102,0.600333,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,253.000000,254.000000,253.000000,79.000000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [0]:
HEIGHT = 28
WIDTH = 28
CHANNELS = 1 # Grayscale

In [14]:
from numpy import reshape

x_mnist_train_small_matrices = reshape(x_mnist_train_small.values, (-1, HEIGHT, WIDTH, CHANNELS))
x_mnist_train_small_matrices.shape

(19999, 28, 28, 1)

Note: [ImageNet images](http://www.image-net.org/challenges/LSVRC/2012/nonpub-downloads) are commonly resized to 224x224

In [15]:
HEIGHT = 224
WIDTH = 224

from numpy import array, float32

from PIL import Image

x_mnist_train_small_scaled = array([array(Image.fromarray(arr[:,:,0]).resize((HEIGHT, WIDTH))).reshape((HEIGHT, WIDTH, CHANNELS)) for arr in x_mnist_train_small_matrices.astype(float32)])
x_mnist_train_small_scaled.shape

(19999, 224, 224, 1)

In [0]:
inputs = x_mnist_train_small_scaled

In [29]:
GIGA = 1<<30

from sys import getsizeof

getsizeof(inputs) / GIGA

3.738216534256935

## ResNet v2

In [16]:
!wget https://github.com/tensorflow/models/blob/master/research/slim/nets/resnet_v2.py


Redirecting output to ‘wget-log.8’.


In [17]:
!ls

mobilenet_v2_1.0_96		   resnet_v2.py.1  wget-log.4
mobilenet_v2_1.0_96.tgz		   resnet_v2.py.2  wget-log.5
mobilenet_v2_1.0_96.tgz.1	   sample_data	   wget-log.6
resnet_v2_101_2017_04_14	   wget-log	   wget-log.7
resnet_v2_101_2017_04_14.tar.gz    wget-log.1	   wget-log.8
resnet_v2_101_2017_04_14.tar.gz.1  wget-log.2
resnet_v2.py			   wget-log.3


In [0]:
from tensorflow.contrib import slim

https://github.com/tensorflow/models/blob/master/research/slim/nets/resnet_v2.py#L29

Typical use:

In [0]:
from tensorflow.contrib.slim.nets import resnet_v2

### [ResNet-101](https://github.com/tensorflow/models/blob/master/research/slim/nets/resnet_v2.py#L274) for image classification into `n_classes` \[1000 for ImageNet\] classes:

In [0]:
from tensorflow import AUTO_REUSE

In [23]:
# inputs has shape [batch, 224, 224, 3]
with slim.arg_scope(resnet_v2.resnet_arg_scope()):
  net, end_points = resnet_v2.resnet_v2_101(inputs, num_classes=n_classes, is_training=False, reuse=AUTO_REUSE)
    
net

ValueError: ignored

In [55]:
logits = Series([net[sample,:,:,:].value_index for sample in range(19999)])
logits.describe()

count    19999.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
dtype: float64

In [58]:
from tensorflow import Session

with Session() as sess:
    net.eval()

FailedPreconditionError: ignored

In [48]:
from tensorflow.losses import log_loss

log_loss(y_mnist_train_small, logits, scope='resnet_v2_101')

<tf.Tensor 'resnet_v2_101_8/value:0' shape=() dtype=float32>

### ResNet-101 for semantic segmentation into 21 classes:

In [32]:
# inputs has shape [batch, 513, 513, 3]
   with slim.arg_scope(resnet_v2.resnet_arg_scope()):
      net, end_points = resnet_v2.resnet_v2_101(inputs,
                                                21,
                                                is_training=False,
                                                global_pool=False,
                                                output_stride=16, reuse=AUTO_REUSE)

ValueError: ignored